In [ ]:
import pandas as pd
import os
from typing import Dict, List, Optional
import numpy as np

# Fungsi 1: Konversi Excel ke CSV dan Load ke Memory
def convert_excel_to_csv_and_load(excel_file_path: str, output_dir: str = 'inventory_csv_data') -> Dict[str, pd.DataFrame]:
    """
    Mengkonversi setiap sheet dalam file Excel menjadi file CSV 
    dan memuatnya ke dalam memory sebagai dictionary of DataFrames.
    
    Args:
        excel_file_path (str): Path ke file .xlsx.
        output_dir (str): Nama folder untuk menyimpan file CSV.
        
    Returns:
        Dict[str, pd.DataFrame]: Dictionary berisi {sheet_name: DataFrame}.
    """
    print("="*60)
    print("🚀 EXCEL TO CSV CONVERTER & DATAFRAME LOADER")
    print("="*60)
    
    # 1. Pastikan file Excel ada
    if not os.path.exists(excel_file_path):
        print(f"❌ Error: File tidak ditemukan di path: {excel_file_path}")
        return {}
    
    # 2. Buat folder output
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)
        print(f"✓ Created folder: {output_dir}")
        
    # 3. Baca semua sheets dari Excel
    try:
        xls = pd.ExcelFile(excel_file_path)
        sheet_names = xls.sheet_names
        print(f"✓ Found {len(sheet_names)} sheets in {os.path.basename(excel_file_path)}")
    except Exception as e:
        print(f"❌ Error reading Excel file: {e}")
        return {}
        
    all_dataframes: Dict[str, pd.DataFrame] = {}
    
    print("\n💾 Processing and Exporting Sheets to CSV...")
    for sheet_name in sheet_names:
        print(f"   Processing sheet: {sheet_name}...", end='\r')
        try:
            # Baca sheet ke DataFrame
            df = xls.parse(sheet_name)
            
            # Simpan ke CSV
            csv_file_path = os.path.join(output_dir, f"{sheet_name}.csv")
            df.to_csv(csv_file_path, index=False)
            
            # Simpan ke dictionary untuk analisis
            all_dataframes[sheet_name] = df
            print(f"   ✅ {sheet_name}: {len(df):,} rows, {len(df.columns)} columns. Exported to {csv_file_path}")
            
        except Exception as e:
            print(f"   ❌ Failed to process sheet {sheet_name}: {e}")
            
    print("\n✅ All Sheets Processed and Loaded into Memory!")
    return all_dataframes

# Fungsi 2: Menampilkan Ringkasan Komprehensif dari DataFrames
def display_comprehensive_dataframe_summary(dataframes: Dict[str, pd.DataFrame]):
    """
    Menampilkan ringkasan terstruktur dari semua DataFrames.
    
    Args:
        dataframes (Dict[str, pd.DataFrame]): Dictionary of DataFrames.
    """
    print("\n" + "="*80)
    print("🔍 COMPREHENSIVE DATAFRAME STRUCTURE ANALYSIS")
    print("="*80)
    
    if not dataframes:
        print("No DataFrames to display.")
        return
        
    for sheet_name, df in dataframes.items():
        print("\n" + "="*80)
        print(f"📊 Sheet: {sheet_name}")
        print("="*80)
        
        total_rows, total_cols = df.shape
        print(f"📏 Shape: {total_rows:,} rows × {total_cols} columns")
        print("-" * 50)
        
        summary_list = []
        for col in df.columns:
            # Hitung statistik dasar
            dtype = df[col].dtype
            non_null_count = df[col].count()
            null_count = total_rows - non_null_count
            null_percentage = (null_count / total_rows) * 100 if total_rows > 0 else 0.0
            
            # Analisis Nilai Unik
            unique_count = df[col].nunique()
            
            # Ambil nilai unik teratas untuk kolom dengan sedikit kategori
            unique_values = "Too many unique values"
            if unique_count <= 20 and unique_count > 0:
                # Mengambil value_counts() dan memformatnya
                top_values = df[col].value_counts().head(5).index.tolist()
                unique_values = str(top_values)
            elif unique_count == 0:
                unique_values = "All Null/Zero"
            elif unique_count == 1:
                unique_values = f"Single Value: {df[col].iloc[0]}"

            # Catat ringkasan
            summary_list.append({
                'Column': col,
                'Type': dtype,
                'Non-Null': f"{non_null_count:,}",
                'Null %': f"{null_percentage:.2f}%",
                'Unique Count': unique_count,
                'Top/Unique Values Sample': unique_values[:100] # Batasi tampilan
            })

        # Konversi ke DataFrame Pandas untuk tampilan yang bagus
        summary_df = pd.DataFrame(summary_list)
        
        # Atur lebar tampilan (opsional, untuk tampilan di console)
        pd.set_option('display.max_rows', None)
        pd.set_option('display.max_columns', None)
        pd.set_option('display.width', 1000)
        pd.set_option('display.colheader_justify', 'left')
        
        # Tampilkan DataFrame
        print(summary_df.to_string())

# ====================================================================
# MAIN EXECUTION
# ====================================================================
if __name__ == "__main__":
    # Ganti dengan path file .xlsx Anda yang sudah di-download
    EXCEL_FILE_PATH = 'sales_performance_analytics.xlsx'
    
    # Folder tempat menyimpan CSV
    CSV_OUTPUT_FOLDER = 'data_project2'
    
    # 1. Konversi dan Load data ke memory
    loaded_dataframes = convert_excel_to_csv_and_load(EXCEL_FILE_PATH, CSV_OUTPUT_FOLDER)
    
    # 2. Tampilkan struktur data yang komprehensif
    display_comprehensive_dataframe_summary(loaded_dataframes)

# Catatan: Jika Anda ingin menyesuaikan kolom yang ditampilkan di summary, 
# ubah list 'summary_list' dan DataFrame 'summary_df' di fungsi 
# 'display_comprehensive_dataframe_summary'.